In [1]:
import os
import cv2
import gc
import numpy as np
import pandas as pd
import itertools
from tqdm import tqdm
#from tqdm.autonotebook import tqdm # if first is not working
import albumentations as A
import matplotlib.pyplot as plt
import torch
from torch import nn
import torch.nn.functional as F
import timm

# for hubert-base-cc
from transformers import AutoModel, AutoTokenizer

# for sentiment-hts2-hubert-hungarian
from transformers import AutoTokenizer, AutoModelForSequenceClassification

# for xlm-roberta-base
from transformers import AutoTokenizer, AutoModelForMaskedLM

import pprint

c:\Users\L0021\Anaconda3\envs\tti\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
image_path = "images_30k"
captions_path = "captions"
file="captions_30k_hu"
csv_name = f"{file}.csv"
df = pd.read_csv(f"{captions_path}/{file}.txt", delimiter="|")
df.columns = ['image', 'caption_number', 'caption']
df['caption'] = df['caption']
df['caption_number'] = df['caption_number']
ids = [id_ for id_ in range(len(df) // 5) for i in range(5)]
df['id'] = ids
df

,image,caption_number,caption,id
0,1000092795.jpg,0,"Két bozontos hajú fiatal srác nézi a kezüket, ...",0
1,1000092795.jpg,1,"Két fiatal, fehér hím van kint sok bokor mellett.",0
2,1000092795.jpg,2,Két zöld inges férfi áll az udvaron.,0
3,1000092795.jpg,3,Egy férfi kék ingben áll a kertben.,0
4,1000092795.jpg,4,Két barát élvezi az együtt töltött időt.,0
...,...,...,...,...
158910,998845445.jpg,0,Egy rövidnadrágos és hawaii inges férfi egy pi...,31782
158911,998845445.jpg,1,"Fiatal férfi lóg egy csónak oldala fölött, ami...",31782
158912,998845445.jpg,2,Egy férfi egy kék - fehér csónak oldaláról dől...,31782
158913,998845445.jpg,3,Egy férfi egy kis csónakon lovagol egy kikötőb...,31782


In [4]:
class CFG:
    debug = False
    image_path = image_path
    captions_path = captions_path
    batch_size = 32
    num_workers = 0 # it was 4 on colab, should be 0 on our pc-s
    head_lr = 1e-3
    image_encoder_lr = 1e-4
    text_encoder_lr = 1e-5
    weight_decay = 1e-3
    patience = 1 # for server running it could be 10, 30 or even 50
    patience_val = 10
    factor = 0.8
    epochs = 1000 # could be 1000
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(device)

    model_name = 'tf_efficientnetv2_b1'
    image_embedding = 1280  # for efficientnetv2b1

    text_model_name = ["SZTAKI-HLT/hubert-base-cc", "xlm-roberta-base", "NYTK/sentiment-hts2-hubert-hungarian"]
    text_encoder_model = text_model_name[2]
    text_embedding = 768
    text_tokenizer = text_encoder_model
    max_length = 200

    pretrained = True   # for both image encoder and text encoder
    trainable = False   # for both image encoder and text encoder
    temperature = 1.0

    # image size
    size = 240  # for efficientnetv2b1

    # for projection head; used for both image and text encoders
    num_projection_layers = 1
    projection_dim = 256 
    dropout = 0.1

cuda


In [12]:
class CLIPDataset(torch.utils.data.Dataset):
    def __init__(self, image_filenames, captions, tokenizer, transforms):
        """
        image_filenames and cpations must have the same length; so, if there are
        multiple captions for each image, the image_filenames must have repetitive
        file names 
        """

        self.image_filenames = image_filenames
        self.captions = list(captions)
        self.encoded_captions = tokenizer(
            list(captions), padding=True, truncation=True, max_length=CFG.max_length
        )
        self.transforms = transforms

    def __getitem__(self, idx):
        item = {
            key: torch.tensor(values[idx])
            for key, values in self.encoded_captions.items()
        }

        image = cv2.imread(f"{CFG.image_path}/{self.image_filenames[idx]}")
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        image = self.transforms(image=image)['image']
        item['image'] = torch.tensor(image).permute(2, 0, 1).float()
        #item['caption'] = self.captions[idx]   # it is not needed, just to help visualize

        return item


    def __len__(self):
        return len(self.captions)



def get_transforms(mode="train"):
    if mode == "train":
        return A.Compose(
            [
                A.Resize(CFG.size, CFG.size, always_apply=True),
                A.Normalize(max_pixel_value=255.0, always_apply=True),
            ]
        )
    else:
        return A.Compose(
            [
                A.Resize(CFG.size, CFG.size, always_apply=True),
                A.Normalize(max_pixel_value=255.0, always_apply=True),
            ]
        )

In [8]:
class ImageEncoder(nn.Module):
    def __init__(
        self, model_name=CFG.model_name, pretrained=CFG.pretrained, trainable=CFG.trainable
    ):
        super().__init__()
        self.model = timm.create_model(
            model_name,
            pretrained, 
            num_classes=0,
            global_pool="avg"
        )
        for p in self.model.parameters():
            p.requires_grad = trainable

    def forward(self, x):
        return self.model(x)

In [9]:
class TextEncoder(nn.Module):
    def __init__(self, model_name=CFG.text_encoder_model, pretrained=CFG.pretrained, trainable=CFG.trainable):
        super().__init__()

        print(model_name)
        if(CFG.text_model_name.index(model_name)==0):
            self.model = AutoModel.from_pretrained(model_name) # hubert-base-cc
        elif(CFG.text_model_name.index(model_name)==1):
            self.model = AutoModelForMaskedLM.from_pretrained(model_name)  # xlm-roberta-base
        elif(CFG.text_model_name.index(model_name)==2):
            self.model = AutoModelForSequenceClassification.from_pretrained(model_name) # sentiment-hts2-hubert-hungarian
            
        for p in self.model.parameters():
            p.requires_grad = trainable

        # we are using the CLS token hidden representation as the sentence's embedding
        self.target_token_idx = 0

    def forward(self, input_ids, attention_mask):

        if(CFG.text_model_name.index(CFG.text_encoder_model)==0):
            output = self.model(input_ids=input_ids, attention_mask=attention_mask)    # hubert-base-cc
            last_hidden_state = output.last_hidden_state   # hubert-base-cc
        else:
            output = self.model(input_ids=input_ids, attention_mask=attention_mask, output_hidden_states=True)    # other
            last_hidden_state = output.hidden_states[-1]   # other
        return last_hidden_state[:, self.target_token_idx, :]

In [10]:
class ProjectionHead(nn.Module):
    def __init__(
        self,
        embedding_dim,
        projection_dim=CFG.projection_dim,
        dropout=CFG.dropout
    ):
        super().__init__()
        self.projection = nn.Linear(embedding_dim, projection_dim)
        self.gelu = nn.GELU()
        self.fc = nn.Linear(projection_dim, projection_dim)
        self.dropout = nn.Dropout(dropout)
        self.layer_norm = nn.LayerNorm(projection_dim)
    
    def forward(self, x):
        projected = self.projection(x)
        x = self.gelu(projected)
        x = self.fc(x)
        x = self.dropout(x)
        x = x + projected
        x = self.layer_norm(x)
        return x

In [11]:
class CLIPModel(nn.Module):
    def __init__(
        self,
        temperature=CFG.temperature,
        image_embedding=CFG.image_embedding,
        text_embedding=CFG.text_embedding,
    ):
        super().__init__()
        self.image_encoder = ImageEncoder()
        self.text_encoder = TextEncoder()
        self.image_projection = ProjectionHead(embedding_dim=image_embedding)
        self.text_projection = ProjectionHead(embedding_dim=text_embedding)
        self.temperature = temperature

    def forward(self, batch):
        # Getting Image and Text Features
        image_features = self.image_encoder(batch["image"])
        text_features = self.text_encoder(
            input_ids=batch["input_ids"], attention_mask=batch["attention_mask"]
        )
        # Getting Image and Text Embeddings (with same dimension)
        image_embeddings = self.image_projection(image_features)
        text_embeddings = self.text_projection(text_features)

        # Calculating the Loss
        logits = (text_embeddings @ image_embeddings.T) / self.temperature
        images_similarity = image_embeddings @ image_embeddings.T
        texts_similarity = text_embeddings @ text_embeddings.T
        targets = F.softmax(
            (images_similarity + texts_similarity) / 2 * self.temperature, dim=-1
        )
        texts_loss = cross_entropy(logits, targets, reduction='none')
        images_loss = cross_entropy(logits.T, targets.T, reduction='none')
        loss =  (images_loss + texts_loss) / 2.0 # shape: (batch_size)
        return loss.mean()


def cross_entropy(preds, targets, reduction='none'):
    log_softmax = nn.LogSoftmax(dim=-1)
    loss = (-targets * log_softmax(preds)).sum(1)
    if reduction == "none":
        return loss
    elif reduction == "mean":
        return loss.mean()

In [5]:
def make_train_valid_dfs():
    dataframe = pd.read_csv(f"{CFG.captions_path}/{csv_name}")
    max_id = dataframe["id"].max() + 1 if not CFG.debug else 100
    image_ids = np.arange(0, max_id)
    np.random.seed(42)
    np.random.shuffle(image_ids)
    #valid_ids = np.random.choice(image_ids, size=int(0.2 * len(image_ids)), replace=False)
    #train_ids = [id_ for id_ in image_ids if id_ not in valid_ids]
    test_ids = image_ids[int(len(image_ids)*0.8):]
    valid_ids = image_ids[int(len(image_ids)*0.6):int(len(image_ids)*0.8)]
    train_ids = image_ids[:int(len(image_ids)*0.6)]
    train_dataframe = dataframe[dataframe["id"].isin(train_ids)].reset_index(drop=True)
    valid_dataframe = dataframe[dataframe["id"].isin(valid_ids)].reset_index(drop=True)
    test_dataframe = dataframe[dataframe["id"].isin(test_ids)].reset_index(drop=True)
    return train_dataframe, valid_dataframe, test_dataframe

In [7]:
def build_loaders(dataframe, tokenizer, mode):
    transforms = get_transforms(mode=mode)
    dataset = CLIPDataset(
        dataframe["image"].values,
        dataframe["caption"].values,
        tokenizer=tokenizer,
        transforms=transforms,
    )
    dataloader = torch.utils.data.DataLoader(dataset,batch_size=CFG.batch_size,num_workers=CFG.num_workers,shuffle=True if mode == "train" else False,)
    return dataloader

In [6]:
def get_image_embeddings(valid_df, model_path):

    tokenizer = AutoTokenizer.from_pretrained(CFG.text_tokenizer)

    valid_loader = build_loaders(valid_df, tokenizer, mode="valid")
    
    model = CLIPModel().to(CFG.device)
    model.load_state_dict(torch.load(model_path, map_location=CFG.device))
    model.eval()
    
    valid_image_embeddings = []
    with torch.no_grad():
        for batch in tqdm(valid_loader):
            image_features = model.image_encoder(batch["image"].to(CFG.device))
            image_embeddings = model.image_projection(image_features)
            valid_image_embeddings.append(image_embeddings)
    return model, torch.cat(valid_image_embeddings)

In [13]:
_, valid_df, test_df = make_train_valid_dfs()

In [ ]:
model_hubert_base_cc, image_embeddings_hubert_base_cc = get_image_embeddings(test_df, "best_models/hubert-base-cc.pt")

In [ ]:
model_sentiment_hts2_hubert_hungarian, image_embeddings_sentiment_hts2_hubert_hungarian = get_image_embeddings(test_df, "best_models/sentiment-hts2-hubert-hungarian.pt")

In [ ]:
model_xlm_roberta_base, image_embeddings_xlm_roberta_base = get_image_embeddings(test_df, "best_models/xlm-roberta-base.pt")